In [ ]:
# -*- coding: utf-8 -*-
!pip install requests beautifulsoup4 mysql-connector-python --quiet

import requests
from bs4 import BeautifulSoup
import mysql.connector
from mysql.connector import Error

class MySQLHelper:
    def __init__(self, host, port, user, password, database):
        self.conn = mysql.connector.connect(
            host=host,
            port=port,
            user=user,
            password=password,
            database=database,
            charset='utf8mb4'
        )
        self.cursor = self.conn.cursor()

    def create_table(self):
        create_table_sql = """
        CREATE TABLE IF NOT EXISTS baidu_hotsearch (
            id INT AUTO_INCREMENT PRIMARY KEY,
            hot_rank INT NOT NULL,
            title VARCHAR(500) NOT NULL,
            url VARCHAR(1000),
            fetched_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
        ) CHARACTER SET = utf8mb4;
        """
        self.cursor.execute(create_table_sql)
        self.conn.commit()

    def insert_item(self, hot_rank, title, url):
        insert_sql = """
        INSERT INTO baidu_hotsearch (hot_rank, title, url)
        VALUES (%s, %s, %s);
        """
        self.cursor.execute(insert_sql, (hot_rank, title, url))
        self.conn.commit()

    def close(self):
        self.cursor.close()
        self.conn.close()


def fetch_baidu_top10():
    url = "https://top.baidu.com/board?tab=realtime"
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
                      "AppleWebKit/537.36 (KHTML, like Gecko) "
                      "Chrome/120.0.0.0 Safari/537.36"
    }
    resp = requests.get(url, headers=headers, timeout=10)
    resp.raise_for_status()
    html = resp.text
    soup = BeautifulSoup(html, 'html.parser')

    results = []
    items = soup.select('div.category-wrap_iQLoo')
    for idx, item in enumerate(items[:10]):
        title_tag = item.select_one('.c-single-text-ellipsis')
        if not title_tag:
            continue
        title = title_tag.get_text(strip=True)
        link_tag = item.select_one('a')
        url_link = link_tag['href'] if (link_tag and link_tag.has_attr('href')) else None

        rank_tag = item.select_one('.hot-index_1Bl1a')
        if rank_tag:
            try:
                rank = int(rank_tag.get_text(strip=True))
            except:
                rank = idx + 1
        else:
            rank = idx + 1

        results.append({'rank': rank, 'title': title, 'url': url_link})
    return results


if __name__ == "__main__":
    hot10 = fetch_baidu_top10()
    print("Fetched:", hot10)

    helper = MySQLHelper(
        host="mysql-dianxiang251126-firewaking-tinkersmith.g.aivencloud.com",
        port=15058,
        user="avnadmin",
        password="password",
        database="defaultdb",
    )
    helper.create_table()

    for e in hot10:
        helper.insert_item(e['rank'], e['title'], e['url'])
    helper.close()
    print("Inserted {} entries".format(len(hot10)))
